<a href="https://colab.research.google.com/github/habib-gm/Deep-Learning-Lab-AAU/blob/master/DL_Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch

In [13]:
class DenseLayer:
  def __init__(self, n_inputs, n_neurons):
    self.weight = 0.01 * torch.rand((n_neurons,n_inputs))
    self.bias = torch.zeros((1,n_neurons))

  def forward(self, inputs):
    self.output = torch.matmul(inputs, self.weight.T) + self.bias
    return self.output

In [14]:
class ActivationSigmoid:
    def forward(self, inputs):
        sigmoid_values = 1 / (1 + torch.exp(-inputs))
        self.output = sigmoid_values
        return self.output
class ActivationReLU:
    def forward(self, inputs):
        self.output = torch.max(torch.tensor(0), inputs)
        return self.output
class ActivationSoftmax:
    def forward(self, inputs):
        exp_values = torch.exp(inputs - torch.max(inputs, axis=1, keepdim=True).values)
        probabilities = exp_values / torch.sum(exp_values, axis=1, keepdim=True)
        self.output = probabilities
        return self.output

In [15]:
def calculate_accuracy(softmax_outputs, class_targets):
    predicted_classes = torch.argmax(softmax_outputs, dim=1)
    if class_targets.dim() == 2:
        class_targets = torch.argmax(class_targets, dim=1)

    accuracy = torch.mean((predicted_classes == class_targets).float()).item()
    return accuracy

In [16]:
class Loss_CategoricalCrossentropy():
    def calculate(self, output, y):
        sample_losses = self._compute_sample_losses(output, y)
        data_loss = torch.mean(sample_losses)
        return data_loss.item()

    def _compute_sample_losses(self, y_pred, y_true):
        y_pred_clipped = torch.clamp(y_pred, 1e-7, 1 - 1e-7)
        correct_confidences = self._get_correct_confidences(y_pred_clipped, y_true)
        negative_log_likelihoods = -torch.log(correct_confidences)
        return negative_log_likelihoods

    def _get_correct_confidences(self, y_pred_clipped, y_true):
        if len(y_true.size()) == 1:
            correct_confidences = y_pred_clipped[torch.arange(y_pred_clipped.size(0)), y_true]
        elif len(y_true.size()) == 2:
            correct_confidences = torch.sum(y_pred_clipped * y_true, dim=1)
        return correct_confidences


In [17]:
inputs = torch.rand(6, 4)
class_targets = torch.tensor([[0,1,1]])
y = torch.rand(6,3)
inputs, y, class_targets

(tensor([[0.1547, 0.7998, 0.2275, 0.1648],
         [0.1951, 0.2508, 0.7026, 0.2315],
         [0.9953, 0.9252, 0.8318, 0.6914],
         [0.5409, 0.8300, 0.8842, 0.4052],
         [0.1934, 0.6670, 0.8213, 0.9419],
         [0.9508, 0.1574, 0.8113, 0.9990]]),
 tensor([[0.4747, 0.7540, 0.1440],
         [0.6050, 0.7086, 0.5753],
         [0.3937, 0.2984, 0.0840],
         [0.3311, 0.5787, 0.2605],
         [0.9290, 0.5016, 0.6382],
         [0.8727, 0.6090, 0.4122]]),
 tensor([[0, 1, 1]]))

In [18]:
#initializing layers
layer1 =  DenseLayer(4, 18)
layer2 = DenseLayer(18,18)
layer3 = DenseLayer(18,18)
output_layer = DenseLayer(18, 3)

In [19]:
#initializing activation functions
relu = ActivationReLU()
sigmoid = ActivationSigmoid()
softmax = ActivationSoftmax()

In [20]:
# inititializing loss function
loss_function = Loss_CategoricalCrossentropy()

In [21]:
# Training....
X = layer1.forward(inputs)
X = relu.forward(X)
X = layer2.forward(X)
X = relu.forward(X)
X = layer3.forward(X)
X = relu.forward(X)
X = output_layer.forward(X)
output = softmax.forward(X)
output

tensor([[0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333]])

In [22]:
loss = loss_function.calculate(output, y)
accuracy = calculate_accuracy(output, class_targets)

print(f"loss: {loss}")
print(f"accuracy: {accuracy}")

loss: 0.7282438278198242
accuracy: 0.0
